A **Injeção de Dependência** é um padrão de design no qual as dependências de um objeto são fornecidas por meio de um mecanismo externo, em vez de serem criadas internamente.


In [5]:
abstract class Investimento {
    constructor(public nome: string, public valorInvestido: number) {}

    abstract calcularRetornoAnual(): number;

    descrever(): string {
        return `Investimento: ${this.nome}, Valor Investido: R$ ${this.valorInvestido}`;
    }
}

class Imovel extends Investimento {
    constructor(
        nome: string,
        valorInvestido: number,
        public aluguelMensal: number
    ) {
        super(nome, valorInvestido);
    }

    calcularRetornoAnual(): number {
        return this.aluguelMensal * 12;
    }

    descrever(): string {
        return `${super.descrever()}, Aluguel Mensal: R$ ${this.aluguelMensal}`;
    }
}

Repository

In [6]:
interface IRepository {
    save() : void;
}

class Repository implements IRepository {
    
    save(investimento: Investimento): void {
        console.log(`Saving investment: ${JSON.stringify(investimento)}`);
    };
}

Service

In [7]:
class ImovelService {
    save(investimento: Imovel): void {
        let imovel = new Repository();
        imovel.save(investimento);
     }
}

Implementação

In [8]:
const imovelParam: Imovel = {
    tipo: "imovel",
    nome: "Apartamento Z",
    valorInvestido: 300000,
    aluguelMensal: 1500
};

let imovel = new ImovelService();
imovel.save(imovelParam);

Saving investment: {"tipo":"imovel","nome":"Apartamento Z","valorInvestido":300000,"aluguelMensal":1500}


Registrar Container de DI

In [9]:
class Container {
    private services = new Map<string, any>();

    register<T>(name: string, implementation: new (...args: any[]) => T): void {
        this.services.set(name, implementation);
    }

    resolve<T>(name: string): T {
        const implementation = this.services.get(name);
        if (!implementation) {
            throw new Error(`Service not found: ${name}`);
        }
        return new implementation();
    }
}

Registrar as classes

In [10]:
const container = new Container();
container.register<IRepository<Imovel>>("ImovelRepository", Repository);
container.register<ImovelService>("ImovelService", ImovelService);

Refatorar ImovelService

In [11]:
class ImovelService {
    private repository: IRepository<Imovel>;

    constructor(repository: IRepository<Imovel>) {
        this.repository = repository;
    }

    save(investimento: Imovel): void {
        this.repository.save(investimento);
    }
}

Implementação

In [12]:
const imovelParam: Imovel = {
    tipo: "imovel",
    nome: "Apartamento W",
    valorInvestido: 300000,
    aluguelMensal: 2000
};

const imovelRepository = container.resolve<IRepository<Imovel>>("ImovelRepository");

const imovelService = new ImovelService(imovelRepository);
imovelService.save(imovelParam);

Saving investment: {"tipo":"imovel","nome":"Apartamento W","valorInvestido":300000,"aluguelMensal":2000}
